In [17]:
import numpy as np
import pandas as pd
import os, re, pickle

In [8]:
file_dir = "Z:\IVLE Identifiable Data\Deidentify"

In [48]:
stud_grades = pd.read_csv(os.path.join(file_dir, 'IVLE-followup-enrollmentsWinter2019 DEID.csv'))
with open('./student_data.pickle', 'rb') as f:
    stud_qts = pickle.load(f)

In [49]:
student_gpa = {}

In [50]:
def get_term(x):
    year = re.findall('(?<=-)\d+', x['AcadYr'])[0]
    if x['AcadTerm'] == 'Fall':
        year = str(int(year) - 1)
    return year+x['AcadTerm']

In [51]:
lt2gpa = {'A+':4.0, 'A':4.0, 'A-':3.7, 'B+':3.3, 'B':3.0, 'B-':2.7, 'C+':2.3, 'C':2.0, 'C-':1.7, 'D+':1.3, 'D':1.0, 'D-':0.7, 'F':0}

f_grades = stud_grades['Grade'].isin(lt2gpa.keys())
f_qrt = (stud_grades['AcadTerm'] != 'Summer')

stud_grades = stud_grades[f_grades & f_qrt]
stud_grades['gpa'] = stud_grades['Grade'].apply(lambda x: lt2gpa[x])
stud_grades['term'] = stud_grades.apply(get_term, axis=1)

In [54]:
for stud in stud_qts:
    
    sid = int(re.findall('\d+', stud)[0])
    
    f1 = (stud_grades['roster_randomid'] == sid)
    f2 = stud_grades['term'].isin(stud_qts[stud])
    stud_rec = stud_grades[f1&f2]
    
    if len(stud_rec) == 0:
        student_gpa[stud] = 0
    else:
        student_gpa[stud] = np.average(stud_rec['gpa'], weights=stud_rec['CourseUnits']) 

In [56]:
with open('./student_gpa.pickle', 'wb') as f:
    pickle.dump(student_gpa, f, protocol=pickle.HIGHEST_PROTOCOL)